In [1]:
import shlex
import os
from subprocess import check_output
import sys
import torch
import numpy
import pyvista as pv

os.chdir(check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii'))
from config.config_parser import read_config
from helpers import *

In [2]:
import mesh_operations
device = get_device()

2020-09-09 06:30:53,324 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [3]:
import ipywidgets as widgets
from IPython.display import display

In [4]:
output_dir = "../pytorch_coma/output/"
experiments = [x for x in os.listdir(output_dir) if os.path.exists(os.path.join(output_dir, x, ".finished"))]
w = widgets.Dropdown(
    value='2020-08-27_04-58-31',
    options=experiments,
    description='Experiment:',
    disabled=False,
)

display(w)

Dropdown(description='Experiment:', index=6, options=('2020-08-27_05-00-23', '2020-08-27_05-00-30', '2020-08-2…

In [5]:
import ExperimentClass
experiment = ExperimentClass.ComaExperiment(os.path.join(output_dir, w.value))
experiment.load_model()

In [19]:
dataset = load_cardiac_dataset(experiment.config)
dataloader = get_loader(dataset.vertices_val, dataset.val_ids, batch_size=8, num_workers=4, shuffle=False)

2020-08-28 13:46:09,188 [ERROR] Unsupported point attribute type: point_data for file: ./template/template.vtk
2020-08-28 13:46:10,635 [INFO] Performing Procrustes analysis
2020-08-28 13:47:05,235 [INFO] Generalized Procrustes analysis performed after 5 iterations
2020-08-28 13:47:07,460 [INFO] Vertices normalized


In [90]:
dataloader = get_loader(dataset.vertices_val, dataset.val_ids, len(dataset.vertices_val), num_workers=4, shuffle=False)

In [91]:
experiment.model.eval()
torch.no_grad()
for mesh, id in dataloader:
    if experiment.model.is_variational:
        mu, log_var = experiment.model.encoder(x=mesh)
        z = mu # coma.sampling(mu, log_var)
    else:
        z = experiment.model.encoder(x=mesh)
    mesh_r = experiment.model.decoder(z)

In [141]:
from ipywidgets import interact, interactive, fixed, interact_manual
def f(x, reconstructed=True, normalized=True):
  _mesh = mesh_r if reconstructed else mesh
  _mesh = _mesh[x].detach().numpy() if normalized else dataset.mean + dataset.std * _mesh[x].detach().numpy()

  plotter = pv.Plotter(notebook=True)
  plotter.add_mesh(_mesh)
  plotter.show(interactive=True)
  plotter.enable()

interact(f, 
  x=widgets.SelectionSlider(options=range(200)),
  #x=widgets.Dropdown(options=range(len(mesh)), description='mesh index:', disabled=False),
  reconstructed=True,
  normalized=True
)

interactive(children=(SelectionSlider(description='x', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

<function __main__.f(x, reconstructed=True, normalized=True)>

SelectionSlider(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,…